In [1]:
from tqdm import tqdm 
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from utils.base import eval_accuracy
from utils.base import get_correct_predictions_subset
from utils.data import create_butterfly_dataset
from utils.data import create_imagenet_dataset
from model.butterfly_classifier import DenseNet121
from algo.attacker import adversarial_generator
from torch.utils.data import DataLoader
import torch.nn.functional as F

# (0) Download Butterfly 

In [ ]:
!kaggle datasets download -d gpiosenka/butterfly-images40-species

In [ ]:
import zipfile
with zipfile.ZipFile('butterfly-images40-species.zip', 'r') as zip_ref:
    zip_ref.extractall('data/butterfly')

# (1) Process Butterfly Data

In [2]:
trainset, testset, normal_mapping, reverse_mapping, sample_img_dataset = create_butterfly_dataset()
assert len(trainset) == 12594, 'Size of train set not match'
assert len(testset) == 500, 'Size of test set not match'

# (2) Import Classifier

In [3]:
model = DenseNet121(num_classes=len(normal_mapping)).to('cuda')
model.load_state_dict(torch.load('./model/states/butterfly_classifier.pth'))

<All keys matched successfully>

# (3) Evaluate Untargeted Adversarial Examples

We also subset the parts where the model could provide correct predictions to attack

In [4]:
model.eval()
accuracy, correct_subset = get_correct_predictions_subset(model, testset, batch_size=100)
print('Accuracy on test set is {:.4f}'.format(accuracy))
print('Number of correctly predicted samples:', len(correct_subset))

Accuracy on test set is 0.9640
Number of correctly predicted samples: 482


# (4) Parameter Definition

In [47]:
# parameters definition
batch_size = 32
query_limit = 30000 # max attack limit 
LO_query_limit = 3000000 # max attack limit 
search_var = 1e-3 # amount to perturb the input image
sample_num = 50 # 2*sample_num for estimating gradient
bound = 0.1 # the l-infinity distance between the adversarial example and the input image
# partial information paramters + lebel-only parameters
epsilon = 0.5 # initial searching range from the target image
delta = 0.01 # rate to decrease epsilon
eta_max = 0.02 # maximum learning rate
eta_min = 0.01 # minimum learning rate
k = 5 # information access
# label-only parameter
mu = 0.001 # radius for sampling ball
m = 50 # 2*number of sample for proxy score
#correct_subset_loader = DataLoader(correct_subset, batch_size = batch_size, shuffle = False)
correct_subset_loader = DataLoader(correct_subset, batch_size = 2, shuffle = False)
lr = 0.01

# (5) Query-Limited Setting

In [9]:
from utils.base import quick_predict

model.to('cuda')
model.eval()
success_count = 0
query_counts = []
adv_images = []
with torch.no_grad():
    for batch in correct_subset_loader:
        images = batch[0]
        target_classes = batch[1].to('cuda')
        adv_image_batch, query_count_batch = adversarial_generator(model, target_classes, images, 
                                                                 search_var, sample_num,
                                                                bound, lr, query_limit)
        query_counts.append(query_count_batch)
        adv_images.append(adv_image_batch)
        
        adv_class = quick_predict(model, adv_image_batch)
        success_count += (adv_class != target_classes).to(float).sum()
    
adv_image_all = torch.concat(adv_images, dim = 0)
query_count_all = torch.concat(query_counts, dim = 0)
torch.save(adv_image_all, "QL_adv_img.pt")
torch.save(query_count_all, "QL_query.pt")
print('Success count is {}, total count is {}, success attack rate is {}'.format(success_count, len(correct_subset), success_count /len(correct_subset) ))

tensor([True, True], device='cuda:0')
tensor(2., device='cuda:0', dtype=torch.float64)


# (6) Partial-Information Setting

In [45]:
from algo.attacker import PIA_adversarial_generator
from algo.defender import PartialInfo
from tqdm import trange

success_count_PIA = 0
query_counts_PIA = []
adv_images_PIA = []

with torch.no_grad():
    for i in trange(len(correct_subset)):
        
        images = correct_subset[i][0].unsqueeze(0)
        # images = batch[0]
        adv_image_batch, query_count_batch = PIA_adversarial_generator(model, images, sample_img_dataset,
                                                                      epsilon, delta, search_var,
                                                                      sample_num, eta_max, eta_min,
                                                                      bound, k, query_limit, label_only = False)
        query_counts_PIA.append(query_count_batch)
        adv_images_PIA.append(adv_image_batch)
        
        
        reach_bound = (query_count_batch < query_limit).to('cuda')
        successful_attack = (quick_predict(model, adv_image_batch) != correct_subset[i][1]).to('cuda')
        success_count_PIA += (successful_attack * reach_bound).to(int).sum()
        
adv_image_all_pia = torch.concat(adv_images_PIA, dim = 0)
query_count_all_pia = torch.concat(query_counts_PIA, dim = 0)
torch.save(adv_image_all_pia, "PIA_adv_img.pt")
torch.save(query_count_all_pia, "PIA_query.pt")
print('Success count is {}, total count is {}, success attack rate is {}'.format(success_count_PIA, len(correct_subset), success_count_PIA /len(correct_subset) ))

  0%|          | 0/482 [00:47<?, ?it/s]

Success count is 1, total count is 482, success attack rate is 0.004149377593360996


# (7) Label-only Setting

In [ ]:
success_count_LO = 0
query_counts_LO = []
adv_images_LO = []

with torch.no_grad():
    for i in trange(2):
        images = correct_subset[i][0].unsqueeze(0)
        adv_image_batch, query_count_batch = PIA_adversarial_generator(model, images, sample_img_dataset,
                                                                      epsilon, delta, search_var,
                                                                      sample_num, eta_max, eta_min,
                                                                      bound, k, query_limit=LO_query_limit, label_only = True, mu = mu, m = m) # different query_limit
        query_counts_LO.append(query_count_batch)
        adv_images_LO.append(adv_image_batch)
        
        reach_bound = (query_count_batch < LO_query_limit).to('cuda')
        successful_attack = (quick_predict(model, adv_image_batch) != correct_subset[i][1]).to('cuda')
        success_count_LO += (successful_attack * reach_bound).to(int).sum()
        break
        
adv_image_all_lo = torch.concat(adv_images_LO, dim = 0)
query_count_all_lo = torch.concat(query_counts_LO, dim = 0)
torch.save(adv_image_all_lo, "LO_adv_img.pt")
torch.save(query_count_all_lo, "LO_query.pt")
print('Success count is {}, total count is {}, success attack rate is {}'.format(success_count_LO, 2, success_count_LO /2 ))

  0%|          | 0/10 [00:00<?, ?it/s]